In [3]:
import pandas as pd
from functools import reduce
import os
import pandas as pd
import glob
import json

## Read File

In [17]:
province_dict = {}
files = glob.glob("Development-of-a-Prototype-Index-and-Interactive-Simulation-System-for-Assessing-Hospital/data/raw/*.xlsx")
files

['Development-of-a-Prototype-Index-and-Interactive-Simulation-System-for-Assessing-Hospital/data/raw\\doctors.xlsx',
 'Development-of-a-Prototype-Index-and-Interactive-Simulation-System-for-Assessing-Hospital/data/raw\\elderly.xlsx',
 'Development-of-a-Prototype-Index-and-Interactive-Simulation-System-for-Assessing-Hospital/data/raw\\equipment.xlsx',
 'Development-of-a-Prototype-Index-and-Interactive-Simulation-System-for-Assessing-Hospital/data/raw\\insurance.xlsx',
 'Development-of-a-Prototype-Index-and-Interactive-Simulation-System-for-Assessing-Hospital/data/raw\\opd_ipd.xlsx']

## Create key

In [18]:
dict_path = "province_dict.json"
if os.path.exists(dict_path):
    with open(dict_path, "r", encoding="utf-8") as f:
        province_dict = json.load(f)
else:
    province_dict = {}

files = glob.glob("Development-of-a-Prototype-Index-and-Interactive-Simulation-System-for-Assessing-Hospital/data/raw/*.xlsx")

for path in files:
    df = pd.read_excel(path)
    col = next((c for c in df.columns if "จังหวัด" in c or "Province" in c), None)
    if not col:
        continue
    for p in df[col].dropna().unique():
        if p not in province_dict:
            province_dict[p] = f"P{len(province_dict)+1000}"
    df["ProvinceKey"] = df[col].map(province_dict)
    df.to_excel(path.replace(".xlsx", "_with_key.xlsx"), index=False)
    print(f"{os.path.basename(path)} done")

doctors.xlsx done
elderly.xlsx done
equipment.xlsx done
insurance.xlsx done
opd_ipd.xlsx done


## Save dictionary

In [19]:
with open(dict_path, "w", encoding="utf-8") as f:
    json.dump(province_dict, f, ensure_ascii=False, indent=2)
print("province_dict.json updated")

province_dict.json updated


## Sum File

In [20]:
path = "Development-of-a-Prototype-Index-and-Interactive-Simulation-System-for-Assessing-Hospital/data/raw"
files = glob.glob(os.path.join(path, "*_with_key.xlsx"))

dfs = []

for f in files:
    df = pd.read_excel(f)
    col = next((c for c in df.columns if "จังหวัด" in c or "Province" in c), None)
    if not col:
        continue
    keep_cols = [col, "ProvinceKey"]
    num_cols = df.select_dtypes(include="number").columns.tolist()
    keep_cols.extend(num_cols)
    df = df[keep_cols]
    source_name = os.path.basename(f).replace("_with_key.xlsx", "")
    df = df.rename(columns={c: f"{source_name}_{c}" for c in num_cols})
    dfs.append(df)

from functools import reduce
merged = reduce(lambda left, right: pd.merge(left, right, on=["ProvinceKey", col], how="outer"), dfs)
merged = merged.sort_values("ProvinceKey").reset_index(drop=True)

print(merged.shape)
merged.tail()

(77, 103)


,จังหวัด,ProvinceKey,doctors_ศัลยศาสตร์,doctors_ประสาทศัลยศาสตร์,doctors_ศัลยศาสตร์ตกแต่ง,doctors_ศัลยศาสตร์ทรวงอก,doctors_ศัลยศาสตร์ยูโรวิทยา,doctors_กุมารศัลยศาสตร์,doctors_ศัลยศาสตร์ลำไส้ใหญ่และทวารหนัก,doctors_ศัลยศาสตร์หลอดเลือด,...,equipment_เครื่องตรวจอวัยวะด้วยสนามแม่เหล็กไฟฟ้า,equipment_เครื่องสลายนิ่ว,equipment_เครื่องอัลตราซาวน์,equipment_เครื่องล้างไต,equipment_รถพยาบาล,equipment_จำนวนเตียง,insurance_ลำดับ,insurance_จำนวนโรงพยาบาล,opd_ipd_จำนวนผู้ป่วยนอกเฉลี่ยต่อวัน,opd_ipd_จำนวนผู้ป่วยในเฉลี่ยต่อวัน
72,ปัตตานี,P1072,7,1,2,0,2,2,0,1,...,1,1,58,56,94,1155,32,1,4695.097902,1044.567123
73,พัทลุง,P1073,6,2,1,0,2,0,0,0,...,1,1,41,42,52,1009,36,1,5183.552448,785.868493
74,ยะลา,P1074,11,4,2,1,4,2,0,1,...,2,3,51,61,49,1122,46,2,5199.671329,970.756164
75,สงขลา,P1075,54,16,5,2,13,4,0,5,...,8,6,130,214,91,4035,58,4,16798.300699,3196.142466
76,สตูล,P1076,4,1,1,0,0,0,0,0,...,1,1,23,11,22,628,59,1,2827.814685,420.106849


In [21]:
merged['จังหวัด'].unique()

array(['กรุงเทพมหานคร', 'เชียงราย', 'เชียงใหม่', 'น่าน', 'พะเยา', 'แพร่',
       'แม่ฮ่องสอน', 'ลำปาง', 'ลำพูน', 'ตาก', 'พิษณุโลก', 'เพชรบูรณ์',
       'สุโขทัย', 'อุตรดิตถ์', 'กำแพงเพชร', 'ชัยนาท', 'นครสวรรค์',
       'พิจิตร', 'อุทัยธานี', 'นครนายก', 'นนทบุรี', 'ปทุมธานี',
       'พระนครศรีอยุธยา', 'ลพบุรี', 'สระบุรี', 'สิงห์บุรี', 'อ่างทอง',
       'กาญจนบุรี', 'นครปฐม', 'ประจวบคีรีขันธ์', 'เพชรบุรี', 'ราชบุรี',
       'สมุทรสงคราม', 'สมุทรสาคร', 'สุพรรณบุรี', 'จันทบุรี', 'ฉะเชิงเทรา',
       'ชลบุรี', 'ตราด', 'ปราจีนบุรี', 'ระยอง', 'สมุทรปราการ', 'สระแก้ว',
       'กาฬสินธุ์', 'ขอนแก่น', 'มหาสารคาม', 'ร้อยเอ็ด', 'นครพนม',
       'บึงกาฬ', 'เลย', 'สกลนคร', 'หนองคาย', 'หนองบัวลำภู', 'อุดรธานี',
       'ชัยภูมิ', 'นครราชสีมา', 'บุรีรัมย์', 'สุรินทร์', 'มุกดาหาร',
       'ยโสธร', 'ศรีสะเกษ', 'อำนาจเจริญ', 'อุบลราชธานี', 'กระบี่',
       'ชุมพร', 'นครศรีธรรมราช', 'พังงา', 'ภูเก็ต', 'ระนอง',
       'สุราษฎร์ธานี', 'ตรัง', 'นราธิวาส', 'ปัตตานี', 'พัทลุง', 'ยะลา',
       'สงขลา', 'สตูล'], 

In [8]:
merged.columns

Index(['จังหวัด', 'ProvinceKey', 'doctors_ศัลยศาสตร์',
       'doctors_ประสาทศัลยศาสตร์', 'doctors_ศัลยศาสตร์ตกแต่ง',
       'doctors_ศัลยศาสตร์ทรวงอก', 'doctors_ศัลยศาสตร์ยูโรวิทยา',
       'doctors_กุมารศัลยศาสตร์', 'doctors_ศัลยศาสตร์ลำไส้ใหญ่และทวารหนัก',
       'doctors_ศัลยศาสตร์หลอดเลือด',
       ...
       'equipment_เครื่องตรวจอวัยวะด้วยสนามแม่เหล็กไฟฟ้า',
       'equipment_เครื่องสลายนิ่ว', 'equipment_เครื่องอัลตราซาวน์',
       'equipment_เครื่องล้างไต', 'equipment_รถพยาบาล', 'equipment_จำนวนเตียง',
       'insurance_ลำดับ', 'insurance_จำนวนโรงพยาบาล',
       'opd_ipd_จำนวนผู้ป่วยนอกเฉลี่ยต่อวัน',
       'opd_ipd_จำนวนผู้ป่วยในเฉลี่ยต่อวัน'],
      dtype='object', length=103)

In [ ]:
# import requests
# import pandas as pd

# url = "https://opendata-service.moph.go.th/gis/v1/getgis/provcode/12"

# # ดึงข้อมูลจาก API
# res = requests.get(url)
# data = res.json()

# # ดึงเฉพาะ features
# features = data["features"]

# # แปลงเป็น DataFrame
# df = pd.json_normalize(features)

# df


,type,geometry.coordinates,geometry.type,properties.bed,properties.distcode,properties.subdistcode,properties.address,properties.provcode,properties.postcode,properties.level_service,properties.hostype,properties.hoscode,properties.moo,properties.hosname,properties.dep
0,Feature,"[100.49238324000000000000, 13.8586378400000000...",Point,0,01,02,ถ.เรวดี,12,11000,10-1 ปฐมภูมิ,18,00993,None,โรงพยาบาลส่งเสริมสุขภาพตำบลตลาดขวัญ,21002
1,Feature,"[100.52141892000000000000, 13.8421013420000000...",Point,0,01,03,2/1,12,11000,10-1 ปฐมภูมิ,18,00995,None,โรงพยาบาลส่งเสริมสุขภาพตำบลบางเขน,21002
2,Feature,"[100.52771866140165000000, 13.8832169162396330...",Point,0,01,05,30/19 ถ.สามัคคี,12,11000,10-1 ปฐมภูมิ,18,00996,None,โรงพยาบาลส่งเสริมสุขภาพตำบลท่าทราย,21002
3,Feature,"[100.52157372000000000000, 13.8724028600000000...",Point,0,01,05,ซ.ทานสัมฤทธิ์ ถ.ติวานนท์,12,11000,10-1 ปฐมภูมิ,18,00997,None,โรงพยาบาลส่งเสริมสุขภาพตำบลทานสัมฤทธิ์,21002
4,Feature,"[100.48731847000000000000, 13.8258409080000000...",Point,0,01,06,33/2 ถ.นครอินทร์,12,11000,10-1 ปฐมภูมิ,18,00998,None,โรงพยาบาลส่งเสริมสุขภาพตำบลวัดโชติการาม,21002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Feature,"[100.50016514745357000000, 13.8618321197633630...",Point,-,01,04,-,12,-,10-1 ปฐมภูมิ,13,15239,None,ศูนย์บริการสาธารณสุขที่ 4 วัดสมรโกฎิ,
83,Feature,"[100.49672583961137000000, 13.8073762337355030...",Point,-,02,02,-,12,-,10-1 ปฐมภูมิ,13,15243,None,ศูนย์บริการสาธารณสุขชุมชนหมู่บ้านสมชาย,21002
84,Feature,"[100.49725585905003000000, 13.9015229465158740...",Point,-,06,02,78/14 ม.1 ซ.สถานสงเคราะห์,12,11000,10-1 ปฐมภูมิ,08,22424,None,ศูนย์สุขภาพชุชนโรงพยาบาลชลประทาน,
85,Feature,"[100.35282806108853000000, 13.9107875291540670...",Point,-,04,04,46/69-72 ถ.บางไผ่-หนองเพรางาย,12,11110,-,16,22604,None,มิตรไมตรีคลินิกเวชกรรม (พฤกษา3),
